# Troppo 오믹스 통합 튜토리얼 (Omics Integration Tutorial)

이 튜토리얼은 Troppo 패키지를 사용하여 오믹스 데이터를 대사 모델에 통합하는 다양한 방법론을 소개합니다.

This tutorial introduces various methodologies for integrating omics data into metabolic models using the Troppo package.

## 목차 (Contents)

1. [환경 설정 및 데이터 로드](#setup)
2. [오믹스 데이터 전처리](#preprocessing)
3. [통합 방법론](#integration-methods)
   - [GIMME](#gimme)
   - [tINIT](#tinit)
   - [iMAT](#imat)
   - [FastCORE](#fastcore)
4. [결과 비교 및 분석](#comparison)

---

## 1. 환경 설정 및 데이터 로드 {#setup}

### 필요한 라이브러리 임포트

In [ ]:
import pandas as pd
import cobra
import re
import numpy as np
from typing import Dict, List

# Troppo imports
from troppo.omics.readers.generic import TabularReader
from troppo.methods_wrappers import ModelBasedWrapper
from troppo.omics.integration import (
    ContinuousScoreIntegrationStrategy,
    ThresholdSelectionIntegrationStrategy,
    DefaultCoreIntegrationStrategy
)

# Reconstruction methods
from troppo.methods.reconstruction.gimme import GIMME, GIMMEProperties
from troppo.methods.reconstruction.tINIT import tINIT, tINITProperties
from troppo.methods.reconstruction.imat import IMAT, IMATProperties
from troppo.methods.reconstruction.fastcore import FASTCORE, FASTCOREProperties

print("라이브러리 임포트 완료 (Libraries imported successfully)")

### GPR 파싱 함수 정의

HumanGEM 모델의 대체 전사체(alternative transcripts)를 처리하기 위한 함수입니다.

In [ ]:
# GPR 파싱을 위한 정규식 패턴
patt = re.compile('__COBAMPGPRDOT__[0-9]{1}')
replace_alt_transcripts = lambda x: patt.sub('', x)

print("GPR 파싱 함수 정의 완료 (GPR parsing function defined)")

### 대사 모델 및 오믹스 데이터 로드

In [ ]:
# 대사 모델 로드 (HumanGEM)
model_path = 'data/HumanGEM_Consistent_COVID19_HAM.xml'
model = cobra.io.read_sbml_model(model_path)
print(f"모델 로드 완료 (Model loaded): {len(model.reactions)} reactions, {len(model.metabolites)} metabolites")

# 오믹스 데이터 로드 (Transcriptomics)
omics_data_path = 'data/Desai-GTEx_ensembl.csv'
omics_data = pd.read_csv(filepath_or_buffer=omics_data_path, index_col=0)
print(f"오믹스 데이터 로드 완료 (Omics data loaded): {omics_data.shape[0]} samples, {omics_data.shape[1]} genes")

# 데이터 미리보기
omics_data.head()

---

## 2. 오믹스 데이터 전처리 {#preprocessing}

### OmicsContainer 생성

`TabularReader`를 사용하여 오믹스 데이터를 Troppo 형식의 컨테이너로 변환합니다.

In [ ]:
# TabularReader로 오믹스 데이터 읽기
reader = TabularReader(
    path_or_df=omics_data,
    nomenclature='ensemble_gene_id',
    omics_type='transcriptomics'
)

# 첫 번째 샘플의 컨테이너 생성
omics_container = reader.to_containers()[0]
print(f"OmicsContainer 생성 완료: {omics_container}")
print(f"샘플 조건 (Sample condition): {omics_container.get_Condition()}")
print(f"유전자 수 (Number of genes): {len(omics_container.get_Data())}")

### ModelBasedWrapper 생성

대사 모델을 Troppo의 재구성 알고리즘에서 사용할 수 있도록 래핑합니다.

In [ ]:
# 모델 래퍼 생성
model_wrapper = ModelBasedWrapper(
    model=model,
    ttg_ratio=9999,
    gpr_gene_parse_function=replace_alt_transcripts
)

print(f"ModelBasedWrapper 생성 완료")
print(f"반응 수 (Number of reactions): {len(model_wrapper.model_reader.r_ids)}")
print(f"대사물질 수 (Number of metabolites): {len(model_wrapper.model_reader.m_ids)}")

### 유전자-반응 매핑 (Gene-Reaction Mapping)

오믹스 데이터의 유전자 발현 값을 GPR(Gene-Protein-Reaction) 규칙을 통해 반응 점수로 변환합니다.

- **AND 규칙**: 최소값 사용 (min function)
- **OR 규칙**: 합산 사용 (sum function)

In [ ]:
# 유전자 발현을 반응 점수로 매핑
data_map = omics_container.get_integrated_data_map(
    model_reader=model_wrapper.model_reader,
    and_func=min,  # AND 규칙: 최소값
    or_func=sum    # OR 규칙: 합산
)

print(f"데이터 매핑 완료 (Data mapping completed)")
print(f"반응 점수 수 (Number of reaction scores): {len(data_map.get_scores())}")

# 반응 점수 샘플 출력
scores_sample = dict(list(data_map.get_scores().items())[:5])
print("\n샘플 반응 점수 (Sample reaction scores):")
for rxn_id, score in scores_sample.items():
    print(f"  {rxn_id}: {score}")

---

## 3. 통합 방법론 (Integration Methods) {#integration-methods}

다양한 오믹스 통합 방법론을 적용하여 조직 특이적 대사 모델을 재구성합니다.

### 3.1 GIMME (Gene Inactivity Moderated by Metabolism and Expression) {#gimme}

GIMME는 연속적인 유전자 발현 값을 사용하여 발현이 낮은 반응의 플럭스를 최소화합니다.

In [ ]:
print("=" * 60)
print("GIMME 알고리즘 실행 (Running GIMME algorithm)")
print("=" * 60)

# 1. 연속 점수 통합 전략 (Continuous scoring)
def score_apply_gimme(reaction_map_scores):
    """NaN 값을 0으로 대체"""
    return {k: 0 if v is None else v for k, v in reaction_map_scores.items()}

continuous_integration = ContinuousScoreIntegrationStrategy(score_apply=score_apply_gimme)
gimme_scores = continuous_integration.integrate(data_map=data_map)

# 2. Biomass 반응의 인덱스 찾기
idx_biomass = model_wrapper.model_reader.r_ids.index('biomass_human')

# 3. GIMME 속성 설정
gimme_properties = GIMMEProperties(
    exp_vector=[v for k, v in gimme_scores.items()],
    obj_frac=0.8,  # 목적 함수의 80% 유지
    objectives=[{idx_biomass: 1}],
    preprocess=True,
    flux_threshold=0.8,
    solver='CPLEX',
    reaction_ids=model_wrapper.model_reader.r_ids,
    metabolite_ids=model_wrapper.model_reader.m_ids
)

# 4. GIMME 실행
gimme_solver = GIMME(
    S=model_wrapper.S,
    lb=model_wrapper.lb,
    ub=model_wrapper.ub,
    properties=gimme_properties
)

gimme_result = gimme_solver.run()

print(f"\nGIMME 결과 (GIMME Results):")
print(f"  선택된 반응 수 (Number of selected reactions): {len(gimme_result)}")
print(f"  원본 모델 대비 (Percentage of original model): {len(gimme_result)/len(model.reactions)*100:.2f}%")

### 3.2 tINIT (Task-driven Integrative Network Inference for Tissues) {#tinit}

tINIT는 조직 특이적 대사 기능(tasks)을 보존하면서 높은 발현을 가진 반응을 우선적으로 선택합니다.

In [ ]:
print("=" * 60)
print("tINIT 알고리즘 실행 (Running tINIT algorithm)")
print("=" * 60)

# 1. Threshold 기반 점수 통합 (발현 값이 높은 반응 선택)
threshold_value = np.percentile([v for v in data_map.get_scores().values() if v is not None], 75)
print(f"사용된 임계값 (Threshold used): {threshold_value:.2f} (75th percentile)")

threshold_integration = ThresholdSelectionIntegrationStrategy(thresholds=threshold_value)
tinit_core_reactions = threshold_integration.integrate(data_map=data_map)

print(f"Core 반응 수 (Number of core reactions): {len(tinit_core_reactions)}")

# 2. tINIT 속성 설정
tinit_properties = tINITProperties(
    core=tinit_core_reactions,
    solver='CPLEX',
    reaction_ids=model_wrapper.model_reader.r_ids,
    metabolite_ids=model_wrapper.model_reader.m_ids
)

# 3. tINIT 실행
tinit_solver = tINIT(
    S=model_wrapper.S,
    lb=model_wrapper.lb,
    ub=model_wrapper.ub,
    properties=tinit_properties
)

tinit_result = tinit_solver.run()

print(f"\ntINIT 결과 (tINIT Results):")
print(f"  선택된 반응 수 (Number of selected reactions): {len(tinit_result)}")
print(f"  원본 모델 대비 (Percentage of original model): {len(tinit_result)/len(model.reactions)*100:.2f}%")

### 3.3 iMAT (Integrative Metabolic Analysis Tool) {#imat}

iMAT는 발현 데이터를 기반으로 반응을 세 그룹(high, moderate, low)으로 분류하고, 발현 패턴과 일치하는 플럭스 분포를 찾습니다.

In [ ]:
print("=" * 60)
print("iMAT 알고리즘 실행 (Running iMAT algorithm)")
print("=" * 60)

# 1. 발현 값의 분위수 계산
score_values = [v for v in data_map.get_scores().values() if v is not None]
high_threshold = np.percentile(score_values, 67)  # 상위 33%
low_threshold = np.percentile(score_values, 33)   # 하위 33%

print(f"High threshold (67th percentile): {high_threshold:.2f}")
print(f"Low threshold (33rd percentile): {low_threshold:.2f}")

# 2. 반응 분류
highly_expressed = set([k for k, v in data_map.get_scores().items() if v is not None and v >= high_threshold])
lowly_expressed = set([k for k, v in data_map.get_scores().items() if v is not None and v <= low_threshold])

print(f"\n높은 발현 반응 (Highly expressed reactions): {len(highly_expressed)}")
print(f"낮은 발현 반응 (Lowly expressed reactions): {len(lowly_expressed)}")

# 3. iMAT 속성 설정
imat_properties = IMATProperties(
    rh=highly_expressed,
    rl=lowly_expressed,
    epsilon=1.0,
    threshold=1e-3,
    solver='CPLEX',
    reaction_ids=model_wrapper.model_reader.r_ids,
    metabolite_ids=model_wrapper.model_reader.m_ids
)

# 4. iMAT 실행
imat_solver = IMAT(
    S=model_wrapper.S,
    lb=model_wrapper.lb,
    ub=model_wrapper.ub,
    properties=imat_properties
)

imat_result = imat_solver.run()

print(f"\niMAT 결과 (iMAT Results):")
print(f"  선택된 반응 수 (Number of selected reactions): {len(imat_result)}")
print(f"  원본 모델 대비 (Percentage of original model): {len(imat_result)/len(model.reactions)*100:.2f}%")

### 3.4 FastCORE {#fastcore}

FastCORE는 core 반응 세트를 보존하면서 일관성 있는 최소 네트워크를 찾습니다.

In [ ]:
print("=" * 60)
print("FastCORE 알고리즘 실행 (Running FastCORE algorithm)")
print("=" * 60)

# 1. Core 반응 선택 (상위 50% 발현)
core_threshold = np.percentile([v for v in data_map.get_scores().values() if v is not None], 50)
print(f"Core threshold (50th percentile): {core_threshold:.2f}")

fastcore_core = set([k for k, v in data_map.get_scores().items() if v is not None and v >= core_threshold])
print(f"Core 반응 수 (Number of core reactions): {len(fastcore_core)}")

# 2. FastCORE 속성 설정
fastcore_properties = FASTCOREProperties(
    core=fastcore_core,
    solver='CPLEX',
    reaction_ids=model_wrapper.model_reader.r_ids,
    metabolite_ids=model_wrapper.model_reader.m_ids
)

# 3. FastCORE 실행
fastcore_solver = FASTCORE(
    S=model_wrapper.S,
    lb=model_wrapper.lb,
    ub=model_wrapper.ub,
    properties=fastcore_properties
)

fastcore_result = fastcore_solver.run()

print(f"\nFastCORE 결과 (FastCORE Results):")
print(f"  선택된 반응 수 (Number of selected reactions): {len(fastcore_result)}")
print(f"  원본 모델 대비 (Percentage of original model): {len(fastcore_result)/len(model.reactions)*100:.2f}%")

---

## 4. 결과 비교 및 분석 {#comparison}

각 방법론으로 재구성된 모델의 크기와 반응 중복을 비교합니다.

In [ ]:
print("=" * 80)
print("결과 비교 (Results Comparison)")
print("=" * 80)

# 결과 요약
results = {
    'GIMME': gimme_result,
    'tINIT': tinit_result,
    'iMAT': imat_result,
    'FastCORE': fastcore_result
}

# 1. 모델 크기 비교
print("\n1. 모델 크기 비교 (Model Size Comparison)")
print("-" * 80)
print(f"{'Method':<15} {'Reactions':<15} {'% of Original':<20}")
print("-" * 80)

for method, result in results.items():
    num_rxns = len(result)
    percentage = num_rxns / len(model.reactions) * 100
    print(f"{method:<15} {num_rxns:<15} {percentage:<20.2f}%")

print("-" * 80)
print(f"{'Original Model':<15} {len(model.reactions):<15} {100.0:<20}%")

# 2. 반응 중복 분석
print("\n2. 반응 중복 분석 (Reaction Overlap Analysis)")
print("-" * 80)

# 반응 ID 세트로 변환
reaction_sets = {}
for method, result in results.items():
    reaction_sets[method] = set([model_wrapper.model_reader.r_ids[i] for i in result])

# 공통 반응 찾기
all_methods_common = set.intersection(*reaction_sets.values())
print(f"\n모든 방법에 공통된 반응 (Common to all methods): {len(all_methods_common)}")

# 쌍별 중복
print("\n쌍별 중복 (Pairwise Overlap):")
methods = list(results.keys())
for i in range(len(methods)):
    for j in range(i+1, len(methods)):
        overlap = len(reaction_sets[methods[i]] & reaction_sets[methods[j]])
        union = len(reaction_sets[methods[i]] | reaction_sets[methods[j]])
        jaccard = overlap / union if union > 0 else 0
        print(f"  {methods[i]} ∩ {methods[j]}: {overlap} reactions (Jaccard: {jaccard:.3f})")

# 3. 시각화를 위한 데이터프레임 생성
comparison_df = pd.DataFrame({
    'Method': list(results.keys()),
    'Number of Reactions': [len(r) for r in results.values()],
    'Percentage': [len(r)/len(model.reactions)*100 for r in results.values()]
})

print("\n3. 요약 테이블 (Summary Table)")
print("-" * 80)
print(comparison_df.to_string(index=False))

print("\n" + "=" * 80)
print("튜토리얼 완료! (Tutorial completed!)")
print("=" * 80)

---

## 추가 분석 (Additional Analysis)

### 재구성된 모델 저장하기

In [ ]:
# GIMME 결과로 새로운 모델 생성 및 저장 예시
def create_tissue_specific_model(original_model, selected_reaction_indices, model_reader):
    """
    선택된 반응으로 조직 특이적 모델 생성
    
    Parameters:
    -----------
    original_model : cobra.Model
        원본 대사 모델
    selected_reaction_indices : list
        선택된 반응의 인덱스 리스트
    model_reader : ModelReader
        모델 정보를 포함한 리더 객체
    
    Returns:
    --------
    cobra.Model
        조직 특이적 모델
    """
    # 선택된 반응 ID 가져오기
    selected_rxn_ids = [model_reader.r_ids[i] for i in selected_reaction_indices]
    
    # 모델 복사
    tissue_model = original_model.copy()
    
    # 선택되지 않은 반응 제거
    reactions_to_remove = [rxn for rxn in tissue_model.reactions if rxn.id not in selected_rxn_ids]
    tissue_model.remove_reactions(reactions_to_remove, remove_orphans=True)
    
    return tissue_model

# GIMME 모델 생성
gimme_model = create_tissue_specific_model(model, gimme_result, model_wrapper.model_reader)
print(f"GIMME 모델: {len(gimme_model.reactions)} reactions, {len(gimme_model.metabolites)} metabolites")

# 모델 저장 (선택사항)
# cobra.io.write_sbml_model(gimme_model, "tissue_specific_model_GIMME.xml")
# print("모델이 저장되었습니다 (Model saved)")

---

## 참고사항 (Notes)

### 각 방법론의 특징:

1. **GIMME**: 
   - 연속적인 발현 값 사용
   - 발현이 낮은 반응의 플럭스를 최소화
   - 빠른 실행 속도

2. **tINIT**:
   - Task 기반 접근
   - 조직 특이적 기능 보존
   - 높은 발현 반응 우선 선택

3. **iMAT**:
   - 발현 데이터 기반 반응 분류
   - 발현 패턴과 플럭스의 일치성 최대화
   - 질적 발현 데이터에 적합

4. **FastCORE**:
   - Core 반응 세트 보존
   - 일관성 있는 최소 네트워크
   - 빠른 속도

### 솔버 설정:

이 튜토리얼은 CPLEX 솔버를 사용합니다. 다른 솔버(GUROBI, GLPK 등)를 사용하려면 각 Properties 객체의 `solver` 파라미터를 변경하세요.

### 데이터 요구사항:

- 대사 모델: SBML 형식
- 오믹스 데이터: CSV, TSV 등 표 형식
- 유전자 ID: 모델과 오믹스 데이터의 nomenclature 일치 필요

---

**Tutorial by Troppo Package**  
For more information, visit: http://troppo-bisbi.readthedocs.io/